In [28]:
import pandas as pd
from threading import Thread
pd.set_option('max_colwidth',500)
import multiprocessing

In [29]:
import sys
sys.path.append("../scripts/")
import clean_text
from importlib import reload
reload(clean_text)
clean_text.clean('@esto es un #tweet. Hola!!😀', filter_stopwords=True)

'@ # tweet hola ! 😀'

In [9]:
tweets = pd.read_csv('../datasets/tweets_202007282056.csv')

In [10]:
df = tweets.copy()
df.index = df.tweet_id.values
print(len(df))
print(df.columns)
df.head()

2240646
Index(['tweet_id', 'conversation_id', 'user_id', 'created_at', 'lang',
       'retweet_count', 'favorite_count', 'reply_count', 'quote_count',
       'is_reply', 'replied_id', 'replied_user_id', 'is_quote', 'quoted_id',
       'full_text', 'source', 'scrapped_at', 'user_mentions_count',
       'hashtags_count', 'urls_count', 'media_count', 'replied_original'],
      dtype='object')


,tweet_id,conversation_id,user_id,created_at,lang,retweet_count,favorite_count,reply_count,quote_count,is_reply,...,is_quote,quoted_id,full_text,source,scrapped_at,user_mentions_count,hashtags_count,urls_count,media_count,replied_original
1000001051574046720,1000001051574046720,1000001051574046720,323956672,2018-05-25 13:10:04.0,es,9,12,1,0,0,...,0,NaN,Feliz día de la Patria! 🇦🇷 https://t.co/XfrTi29VcC,Twitter for iPhone,2020-07-28 05:10:54.0,0,0,0,1,NaN
1000002592955609088,1000002592955609088,1000002592955609088,53851352,2018-05-25 13:16:11.0,es,596,1486,2,4,0,...,0,NaN,"No busquen explicación, es amor.\n#117AñosDeHistoria https://t.co/b2lueUKodj",Twitter for Android,2020-07-28 04:26:30.0,0,1,0,1,NaN
1000002668474101761,1000002668474101761,1000002668474101761,69668594,2018-05-25 13:16:29.0,es,5,23,0,0,0,...,0,NaN,🇦🇷 La Patria somos todos y la construimos juntos. #FelizDíaDeLaPatria #25deMayo https://t.co/8IK54CBtoW,Twitter for Android,2020-07-28 02:21:15.0,0,2,0,1,NaN
1000003177448697857,1000003177448697857,1000003177448697857,834702151,2018-05-25 13:18:31.0,es,9897,18496,141,691,0,...,0,NaN,- Sos friolenta?\n- https://t.co/qkQNmQTwxE,TweetDeck,2020-07-28 12:31:42.0,0,0,0,1,NaN
1000003312597585921,1000003312597585921,1000003312597585921,971923363038973952,2018-05-25 13:19:03.0,es,1,7,0,0,0,...,0,NaN,Feliz Dia de la Patria! ❤ https://t.co/dqSyZOLt4p,Twitter for Android,2020-07-28 17:33:35.0,0,0,0,1,NaN


In [23]:
print(len(df[(df.favorite_count >= 5) & (df.lang == 'es')]))
print(len(df[df.lang=='es']))
df.sample(1)[['tweet_id', 'full_text']].values[0]

1118383
2058360


array([1184897776775245825,
       'Hoy, a partir de las 18 hs, vas a poder seguir en vivo a través de nuestras redes el acto nacional por el #DíaDeLaLealtad en La Pampa junto a los compañeros y compañeras del Frente de Todos ✌️🏻 https://t.co/fTxrlBEcyn'],
      dtype=object)

In [32]:
from joblib import Parallel, delayed

def paralell_func(values, func, frac=8):
    r = Parallel(n_jobs=8, backend='threading')(delayed(func)(i) for i in values)
    return r

def clean_fn(row):
    row[1] = clean_text.clean(row[1], filter_stopwords=True)
    return row

cleaned_texts = paralell_func(df.sample(5)[['tweet_id', 'full_text']].values, clean_fn)

In [33]:
list(zip(*cleaned_texts))

[(1287540392578121733,
  1283554481691664391,
  958365085604466690,
  1140709720728227846,
  1278471633494302722),
 ('no deja ser buena noticia tampoco significa debamos relajarnos',
  'puré mayonesa manjar dioses',
  'todas medidas contexto déficit presupuestario previo paritarias orden 2 441 millones marco ahorros despidos reducción servicios presta no resuelto camino',
  'terminando primera jornada ! excelente ! @ goglobalsrl @ ferraripau @ _ceaes # acelerarespaña URL',
  'siguen éxitos # gobiernodecientificos @ cfkargentina @ alferdez si culpa pandemia no cuarentena eligen radicarse brasil ? URL')]

In [24]:
df = df[(df.favorite_count >= 5) & (df.lang == 'es')]

In [34]:
frac = multiprocessing.cpu_count()
values = df[['tweet_id', 'full_text']].values
cleaned_texts = paralell_func(values, clean_fn, frac)

In [35]:
to_insert = list(zip(*cleaned_texts))
df.loc[to_insert[0], 'full_text_cleaned'] = to_insert[1]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [36]:
df.head(20)[['full_text_cleaned']]

,full_text_cleaned
1000001051574046720,feliz día patria ! 🇦 🇷 URL
1000002592955609088,no busquen explicación amor # 117añosdehistoria URL
1000002668474101761,🇦 🇷 patria construimos juntos # felizdíadelapatria # 25demayo URL
1000003177448697857,sos friolenta ? URL
1000003312597585921,feliz dia patria ! URL
1000003529396883456,participando tedeum # 25demayo parroquia santa maría carmen # necochea URL
1000004585124253699,feliz día patria ! día comenzamos ser libres URL
1000004674483900418,muchas gracias vecinos acompañaron tradicional toque diana comenzamos festejos mendoza 208 años revolución # 25demayo URL
1000005932535959553,hoy día importante provincia día patria inaguramos autovía rn16 peaje makallé ahora @ radioprovchaco
1000006106494750720,quiénes violentos ilegales subte ? videla macri breve historia grupo empresario ataca metrodelegados URL @ patriciodc @ myriambregman @ nicolasdelcano @ lupo55 @ verbitskyh


In [37]:
df.to_pickle('../datasets/tweets_cleaned.pkl')

In [18]:
df = pd.read_pickle('../datasets/tweets_cleaned.pkl')

In [15]:
len(df[(df.favorite_count >= 5) & (df.lang == 'es')])

1118383

In [19]:
# df.full_text = df.full_text.apply(lambda x: '<s>'+x+'</s>')
df.head().full_text

1000001051574046720                                                         feliz día de la patria ! 🇦 🇷 URL
1000002592955609088                                   no busquen explicación es amor # 117añosdehistoria URL
1000002668474101761    🇦 🇷 la patria somos todos y la construimos juntos # felizdíadelapatria # 25demayo URL
1000003177448697857                                                                      sos friolenta ? URL
1000003312597585921                                                             feliz dia de la patria ! URL
Name: full_text, dtype: object

In [20]:
corpus_to_save = df[(df.favorite_count >= 5) & (df.lang == 'es')].full_text.values
with open("../datasets/tweets_corpus_bert.txt", "w", encoding='utf-8') as text_file:
    text_file.write(
        "\n".join(corpus_to_save)
    )